In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/AN2DL/Homework2

In [2]:
!wget -q https://storage.googleapis.com/storage.barbiero.dev/AN2DL/Homework_2/training_data_clean.npy
!wget -q https://storage.googleapis.com/storage.barbiero.dev/AN2DL/Homework_2/categories_clean.npy
!wget -q https://storage.googleapis.com/storage.barbiero.dev/AN2DL/Homework_2/series_length_clean.npy

In [ ]:
!pip install --upgrade tensorflow -q

In [4]:
# Fix randomness and hide warnings
seed = 42

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np

np.random.seed(seed)

import logging

import random

random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [6]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler

In [7]:
use_tpu = True  #@param {type:"boolean"}

if 'COLAB_TPU_ADDR' in os.environ and use_tpu:
    TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
    use_tpu = False

if use_tpu:
    tpu_address = TF_MASTER
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)

In [8]:
def print_sequence(dataset, to_plot):
    figs, axs = plt.subplots(to_plot, 1, sharex=True, figsize=(17, 17))
    n_elements = len(dataset)
    for i in range(to_plot):
        el = random.randrange(n_elements)
        axs[i].plot(dataset[el])
        axs[i].set_title(el)
    plt.show()

In [9]:
def build_sequence(data, categories, window_size=200, stride=200, telescope=18):
    actual_window = window_size + telescope

    new_categories = []
    X = []
    y = []

    for i, element in enumerate(data):
        length = len(element)
        # number of windows
        n_windows = int(np.ceil((length - actual_window) / stride)) + 1
        # reevaluate the stride
        if n_windows > 1:
            stride = int((length - actual_window) / (n_windows - 1)) + 1
        # start from the end of the series
        start_idx = length - actual_window
        end_idx = length
        # for each window
        for j in range(n_windows):
            if start_idx < 0:
                start_idx = 0
                end_idx = actual_window
                if end_idx > length:
                    end_idx = length
            # append the window to X
            temp = element[start_idx:end_idx - telescope]
            if len(temp) < window_size:
                temp = np.pad(temp, (0, window_size - len(temp)), mode="constant", constant_values=-1)
            X.append(temp)
            # append the telescope to y
            y.append(element[end_idx - telescope:end_idx])
            # append the category
            new_categories.append(categories[i])
            # update the start and end index
            start_idx -= stride
            end_idx -= stride

    return X, y, new_categories

In [ ]:
training_data = np.load("training_data_clean.npy", allow_pickle=True)
training_data.shape

In [ ]:
print_sequence(training_data, 10)

In [13]:
categories = np.load("categories_clean.npy", allow_pickle=True)
series_lengths = np.load("series_length_clean.npy", allow_pickle=True)
categories_Set = set(categories)

dataset = [training_data[i, :series_lengths[i]] for i in range(len(training_data))]

In [ ]:
print_sequence(dataset, 10)

In [15]:
window = 200
stride = 200
telescope = 18
autoregressive_telescope = 18
X, y, categories = build_sequence(dataset, categories, window_size=200, stride=200, telescope=18)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test, categories_train_val, categories_test = train_test_split(X, y, categories,
                                                                                       test_size=0.15,
                                                                                       stratify=categories)
X_train, X_val, y_train, y_val, categories_train, categories_val = train_test_split(X_train_val, y_train_val, categories_train_val,
                                                                                       test_size=0.15,
                                                                                       stratify=categories_train_val)
print(len(X_train))
print(len(X_test))

In [19]:
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
y_val = np.expand_dims(y_val, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [22]:
# Assign the batch size
if use_tpu:
    BATCH_SIZE = 64
else:
    BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

In [23]:
def prepare_trainset(dataset):
    return (
        dataset
        .cache()
        .shuffle(1000, reshuffle_each_iteration=True)
        .repeat()
        .batch(BATCH_SIZE, drop_remainder=True)
    )


def prepare_valset(dataset):
    return (
        dataset
        .cache()
        .repeat()
        .batch(BATCH_SIZE, drop_remainder=True)
    )


train_dataset = prepare_trainset(train_dataset)
val_dataset = prepare_valset(val_dataset)

In [24]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

In [25]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Ensure the input time steps are at least as many as the output time steps
    assert input_shape[0] >= output_shape[0], "For this exercise we want input time steps to be >= of output time steps"

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    x = tfkl.Masking(mask_value=-1)(input_layer)
    # Add a Bidirectional LSTM layer with 64 units
    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True, name='lstm'), name='bidirectional_lstm')(x)

    x = tfkl.Bidirectional(tfkl.LSTM(256, return_sequences=True, name='lstm'), name='bidirectional_lstm_2')(x)

    # Add a 1D Convolution layer with 128 filters and a kernel size of 3
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv')(x)

    # Add a final Convolution layer to match the desired output shape
    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same', name='output_layer')(x)

    # Calculate the size to crop from the output to match the output shape
    crop_size = output_layer.shape[1] - output_shape[0]
    print(output_layer.shape[1])

    # Crop the output to the desired length
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

In [ ]:
if use_tpu:
    with strategy.scope():
        model = build_CONV_LSTM_model(input_shape, output_shape)
else:
    model = build_CONV_LSTM_model(input_shape, output_shape)

In [ ]:
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=300,
    validation_data=val_dataset,
    validation_steps=len(X_val) // BATCH_SIZE,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True,
                                    min_delta=0.0005),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17, 4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18, 3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
reg_predictions = np.array([])
X_temp = X_test
for reg in range(0, telescope, autoregressive_telescope):
    pred_temp = model.predict(X_temp, verbose=0)
    if (len(reg_predictions) == 0):
        reg_predictions = pred_temp
    else:
        reg_predictions = np.concatenate((reg_predictions, pred_temp), axis=1)
    X_temp = np.concatenate((X_temp[:, autoregressive_telescope:, :], pred_temp), axis=1)

In [ ]:
# Print the shape of the predictions
print(f"Predictions shape: {reg_predictions.shape}")

print("Prediction at 18:")
# Calculate and print Mean Squared Error (MSE)
mean_squared_error = tfk.metrics.mean_squared_error(y_test.flatten(), reg_predictions.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")

# Calculate and print Mean Absolute Error (MAE)
mean_absolute_error = tfk.metrics.mean_absolute_error(y_test.flatten(), reg_predictions.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")

print("Prediction at 9:")
y_test_9 = y_test[:, :9]
reg_predictions_9 = reg_predictions[:, :9]

# Calculate and print Mean Squared Error (MSE)
mean_squared_error = tfk.metrics.mean_squared_error(y_test_9.flatten(), reg_predictions_9.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")

# Calculate and print Mean Absolute Error (MAE)
mean_absolute_error = tfk.metrics.mean_absolute_error(y_test_9.flatten(), reg_predictions_9.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")

In [ ]:
def inspect_multivariate_prediction(X, y, pred, telescope, to_print):
    figs, axs = plt.subplots(to_print, 1, sharex=True, figsize=(17, 17))
    n_elements = X.shape[0]
    for i in range(to_print):
        el = random.randrange(n_elements)
        axs[i].plot(np.arange(len(X[el, :, 0])), X[el, :, 0])
        axs[i].plot(np.arange(len(X[el, :, 0]) - 1, len(X[el, :, 0]) + telescope - 1), y[el, :, 0], color='orange')
        axs[i].plot(np.arange(len(X[el, :, 0]) - 1, len(X[el, :, 0]) + telescope - 1), pred[el, :], color='green')
        axs[i].set_ylim(0, 1)
    plt.show()

In [ ]:
inspect_multivariate_prediction(X_test, y_test, reg_predictions, telescope, 10)

In [ ]:
NAME_MODEL = "BidirectionalGRU"
if use_tpu:
    # save model locally from tpu using Tensorflow's "SavedModel" format
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    model.save(NAME_MODEL, options=save_locally)
else:
    model.save(NAME_MODEL)